In [25]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [26]:
# KOBIS 역대 영화 순위 파일 읽어오기 (수정할 때마다 각자 경로에 맞게 바꾸고 진행해야할 것 같습니다.)
datas = pd.read_excel('KOBIS_datas.xlsx', engine='openpyxl', header=4)

# 필요한 컬럼만 추출하기
datas = datas[['영화명', '개봉일', '관객수', '배급사']][:-1]
datas.head()

,영화명,개봉일,관객수,배급사
0,명량,2014-07-30,17616299,(주)씨제이이엔엠
1,극한직업,2019-01-23,16266480,(주)씨제이이엔엠
2,신과함께-죄와 벌,2017-12-20,14414658,롯데쇼핑㈜롯데엔터테인먼트
3,국제시장,2014-12-17,14265682,(주)씨제이이엔엠
4,어벤져스: 엔드게임,2019-04-24,13977602,월트디즈니컴퍼니코리아 유한책임회사


#### 크롤링

In [47]:
'''
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver = webdriver.Chrome("./chromedriver.exe")
driver.get(kobis)

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'td_movie')))

# 관람연령 리스트 생성
ages_list = []

# 총 200개의 영화
# tr_0 부터 tr_199까지 각 영화 하나에 대한 상세 정보가 있음.
for num in range(200):
    trs_searcher = f'//tr[@id="tr_{num}"]'
    # trs_searcher = f'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]'

    trs = driver.find_elements(By.XPATH, trs_searcher)

    # 모든 trs를 순회하면서 관람등급 가져오기
    for tr in trs:
        # 링크 클릭                             
        movie_link = tr.find_element(By.XPATH, './/*[@id="td_movie"]/span/a') # 상세 정보 팝업 접근
        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)
        movie_link.click() # 클릭해서 팝업 열기
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # [관람등급]에 해당하는 텍스트를 출력
        strong_tags = soup.find_all('strong')
        for strong_tag in strong_tags:
            if strong_tag.text == '[관람등급]':
                rating_text = strong_tag.next_sibling.strip()
                ages_list.append(rating_text) # 리스트에서 순서대로 추가
                print(num, rating_text) #tr_0영화의 관람등급이 15세이상관람가인경우 결과는 '0, 15세이상관람가'
                # len(ages_list)가 248인 이유는 중간중간 관람등급 정보가 중복되어 들어가기 때문 -> 해결책 필요 

        # 창 닫기 (이전 창으로 돌아가기)   
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]/span').click()
        time.sleep(2)
'''

'\n# 페이지 접근\nkobis = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"\ndriver = webdriver.Chrome("./chromedriver.exe")\ndriver.get(kobis)\n\n# 페이지 대기\nwait = WebDriverWait(driver, 10)\nwait.until(EC.presence_of_element_located((By.ID, \'td_movie\')))\n\n# 관람연령 리스트 생성\nages_list = []\n\n# 총 200개의 영화\n# tr_0 부터 tr_199까지 각 영화 하나에 대한 상세 정보가 있음.\nfor num in range(200):\n    trs_searcher = f\'//tr[@id="tr_{num}"]\'\n    # trs_searcher = f\'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]\'\n\n    trs = driver.find_elements(By.XPATH, trs_searcher)\n\n    # 모든 trs를 순회하면서 관람등급 가져오기\n    for tr in trs:\n        # 링크 클릭                             \n        movie_link = tr.find_element(By.XPATH, \'.//*[@id="td_movie"]/span/a\') # 상세 정보 팝업 접근\n        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)\n        movie_link.click() # 클릭해서 팝업 열기\n        time.sleep(2)\n\n        html = driver.page_source\n        soup = BeautifulSoup(html, \'h

휘민님이 진행해주신 크롤링 방법으로 [관람등급]을 리스트로 append했더니 중복되는 데이터가 발생했습니다.   
그래서 아래의 방법을 사용해서 다른 방식으로 ages에 append했고,   
같은 방식으로 상영시간과 장르도 리스트화 하여 datas 데이터프레임에 새로운 컬럼으로 추가했습니다.   

In [27]:
# 드라이버 버전 업데이트에 문제가 있어 추가한 코드입니다 무시하셔도 됩니다!
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [28]:
# 드라이버 버전 업데이트에 문제가 있어 추가한 코드입니다 무시하셔도 됩니다!
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [37]:
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(kobis)

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'td_movie')))

# 관람연령 리스트 생성
ages = []

# 총 200개의 영화
# tr_0 부터 tr_199까지 각 영화 하나에 대한 상세 정보가 있음.
for num in range(200):
    trs_searcher = f'//tr[@id="tr_{num}"]'
    # trs_searcher = f'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]'

    trs = driver.find_elements(By.XPATH, trs_searcher)

    # 모든 trs를 순회하면서 관람등급 가져오기
    for tr in trs:
        # 링크 클릭                             
        movie_link = tr.find_element(By.XPATH, './/*[@id="td_movie"]/span/a') # 상세 정보 팝업 접근
        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)
        movie_link.click() # 클릭해서 팝업 열기
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # [관람연령]
        info_total = driver.find_elements(By.XPATH, "//dl[@class='ovf cont']")

        for each in info_total:
            # dl 태그 내 dd 요소 모두 찾기
            dd_total = each.find_elements(By.TAG_NAME, 'dd')

            # 초 단위는 제거하고 분 단위만 가져옴
            age_limit = dd_total[3].text.split('|')[4].strip()
            ages.append(age_limit)

        # 창 닫기 (이전 창으로 돌아가기)   
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]/span').click()
        time.sleep(2)

In [17]:
# 크롤링 양이 방대해서 [관람등급] 따로, [러닝타임]과 [장르]를 따로 실행함

#러닝타임, 장르 리스트 생성
running_time = []
genre = []

for num in range(200):
    trs_searcher = f'//tr[@id="tr_{num}"]'
    # trs_searcher = f'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]'

    trs = driver.find_elements(By.XPATH, trs_searcher)

    # 모든 trs를 순회하면서 관람등급 가져오기
    for tr in trs:
        # 링크 클릭                             
        movie_link = tr.find_element(By.XPATH, './/*[@id="td_movie"]/span/a') # 상세 정보 팝업 접근
        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)
        movie_link.click() # 클릭해서 팝업 열기
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # [러닝타임]
        info_total = driver.find_elements(By.XPATH, "//dl[@class='ovf cont']")

        for each in info_total:
            # dl 태그 내 dd 요소 모두 찾기
            dd_total = each.find_elements(By.TAG_NAME, 'dd')

            # 초 단위는 제거하고 분 단위만 가져옴
            rt_min = dd_total[3].text.split('|')[3].strip().split(' ')[0].strip()
            running_time.append(rt_min)
        
        # [장르]
        for each in info_total:
            # dl 태그 내 dd 요소 모두 찾기
            dd_total = each.find_elements(By.TAG_NAME, 'dd')
            genre_each = dd_total[3].text.split('|')[2].strip()
            genre.append(genre_each)
        
        # 창 닫기 (이전 창으로 돌아가기)   
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]/span').click()
        time.sleep(2)
        
# WebDriver 종료
driver.quit()

In [31]:
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(kobis)

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'td_movie')))

actor_num = []
for num in range(200):
    trs_searcher = f'//tr[@id="tr_{num}"]'
    # trs_searcher = f'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]'

    trs = driver.find_elements(By.XPATH, trs_searcher)

    # 모든 trs를 순회하면서 주연배우 가져오기
    for tr in trs:
        # 링크 클릭                             
        movie_link = tr.find_element(By.XPATH, './/*[@id="td_movie"]/span/a') # 상세 정보 팝업 접근
        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)
        movie_link.click() # 클릭해서 팝업 열기
        time.sleep(0.5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # [출연 배우 정보]
        actor_info_total = driver.find_elements(By.XPATH, '//*[@id="20129370_staff"]/dl') 
        elements = driver.find_elements(By.XPATH, '//td[contains(., "[주연]")]') # [주연] 텍스트를 포함하는 td 태그만 가져오기
        count = 0

        # "|" 기준으로 텍스트 나누어서 주연 배우수 가져오기
        for element in elements:
            text = element.text
            count += text.count("|") + 1
            
        actor_num.append(count)

        # 창 닫기 (이전 창으로 돌아가기)   
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]/span').click()
        time.sleep(0.5)
        
# WebDriver 종료
driver.quit()

In [38]:
len(ages), len(running_time), len(genre), len(actor_num)

(200, 200, 200, 200)

In [39]:
datas['관람등급'] = ages
datas['상영시간'] = running_time
datas['장르'] = genre
datas['주연배우수'] = actor_num
datas

,영화명,개봉일,관객수,배급사,관람등급,상영시간,장르,주연배우수
0,명량,2014-07-30,17616299,(주)씨제이이엔엠,15세이상관람가,128분,"사극, 액션",3
1,극한직업,2019-01-23,16266480,(주)씨제이이엔엠,15세이상관람가,111분,코미디,5
2,신과함께-죄와 벌,2017-12-20,14414658,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,139분,"판타지, 드라마",4
3,국제시장,2014-12-17,14265682,(주)씨제이이엔엠,12세이상관람가,126분,드라마,4
4,어벤져스: 엔드게임,2019-04-24,13977602,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,180분,"액션, SF",12
...,...,...,...,...,...,...,...,...
195,82년생 김지영,2019-10-23,3679265,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,118분,드라마,3
196,분노의 질주: 더 익스트림,2017-04-12,3659683,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,136분,"액션, 범죄, 스릴러",7
197,분노의 질주: 홉스&쇼,2019-08-14,3657627,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,135분,액션,4
198,모가디슈,2021-07-28,3619496,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,121분,"액션, 드라마",6


In [40]:
datas.to_csv('TOP200_movies_info.csv')

In [41]:
datas = pd.read_csv('./TOP200_movies_info.csv', index_col=0)
datas

,영화명,개봉일,관객수,배급사,관람등급,상영시간,장르,주연배우수
0,명량,2014-07-30,17616299,(주)씨제이이엔엠,15세이상관람가,128분,"사극, 액션",3
1,극한직업,2019-01-23,16266480,(주)씨제이이엔엠,15세이상관람가,111분,코미디,5
2,신과함께-죄와 벌,2017-12-20,14414658,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,139분,"판타지, 드라마",4
3,국제시장,2014-12-17,14265682,(주)씨제이이엔엠,12세이상관람가,126분,드라마,4
4,어벤져스: 엔드게임,2019-04-24,13977602,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,180분,"액션, SF",12
...,...,...,...,...,...,...,...,...
195,82년생 김지영,2019-10-23,3679265,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,118분,드라마,3
196,분노의 질주: 더 익스트림,2017-04-12,3659683,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,136분,"액션, 범죄, 스릴러",7
197,분노의 질주: 홉스&쇼,2019-08-14,3657627,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,135분,액션,4
198,모가디슈,2021-07-28,3619496,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,121분,"액션, 드라마",6
